In [43]:
from langchain.chains import LLMChain, SequentialChain
# from langchain.prompts import PromptTemplate
from langchain_aws import ChatBedrock
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [55]:
# 2) LLM: Bedrock (ensure region uses hyphens and, if required, an inference profile model_id)
bedrock_llm = ChatBedrock(
    model_id="us.anthropic.claude-sonnet-3-7-20250219-v1:0",  # inference profile ID or ARN if on-demand isn’t supported
    region_name="us-east-1",
)


In [56]:
# New problem: Developing sustainable energy solutions

# Step 1: Ideation
idea_template = """
Step 1:
We are facing challenges with sustainable energy. I need to brainstorm three innovative energy solutions. Consider factors such as environmental impact, scalability, and cost-effectiveness.
A:
"""


# 1) Prompt: use the variables you declare
idea_prompt = PromptTemplate.from_template(idea_template)

# 3) Runnable pipeline = prompt -> llm -> parser
chain = idea_prompt | bedrock_llm | StrOutputParser()

In [57]:
# Step 2: Evaluation
evaluation_template = """
Step 2:
Evaluate the potential of the three proposed energy solutions. Discuss their environmental friendliness, feasibility, technological requirements, and long-term benefits. Give a success probability for each.
{ideas}
A:
"""

evaluation_prompt = PromptTemplate(
    input_variables=["ideas"],
    template=evaluation_template
)

evaluation_chain = LLMChain(
    llm=bedrock_llm,
    prompt=evaluation_prompt,
    output_key="evaluations"
)

In [58]:
# Step 3: Strategy Development
strategy_template = """
Step 3:
Develop detailed strategies for implementing the energy solutions. Include necessary resources, potential partnerships, and ways to overcome challenges. Also, assess possible unexpected outcomes.
{evaluations}
A:
"""

strategy_prompt = PromptTemplate(
    input_variables=["evaluations"],
    template=strategy_template
)

strategy_chain = LLMChain(
    llm=bedrock_llm,
    prompt=strategy_prompt,
    output_key="strategies"
)

In [59]:
# Step 4: Final Recommendations
recommendation_template = """
Step 4:
Rank the energy solutions based on the analysis. Provide a rationale for each ranking, considering factors like impact, feasibility, and sustainability.
{strategies}
A:
"""

recommendation_prompt = PromptTemplate(
    input_variables=["strategies"],
    template=recommendation_template
)

recommendation_chain = LLMChain(
    llm=bedrock_llm,
    prompt=recommendation_prompt,
    output_key="final_recommendations"
)

In [60]:
# Combine the chains into a sequential process
sustainable_energy_chain = SequentialChain(
    chains=[idea_chain, evaluation_chain, chain, recommendation_chain],
    input_variables=["challenge", "considerations"],
    output_variables=["final_recommendations"],
    verbose=True
)

# Example use case
print(sustainable_energy_chain({
    "challenge": "sustainable energy solutions", 
    "considerations": "environmental impact, scalability, cost-effectiveness"
}))

AttributeError: 'RunnableSequence' object has no attribute 'input_keys'

In [62]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # inference profile ID/ARN
    region_name="us-east-1",
    temperature=0.7,
    max_tokens=512,
)

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "Challenge: {challenge}\nConsider: {considerations}\nGive me 3 solutions."
)

chain = prompt | llm | StrOutputParser()

out = chain.invoke({
    "challenge": "sustainable energy solutions",
    "considerations": "environmental impact, scalability, cost-effectiveness",
})
print(out)

# Sustainable Energy Solutions

## 1. Grid-Scale Battery Storage Systems
- **Environmental Impact**: Enables higher renewable energy integration, reducing fossil fuel dependence; concerns include mining for materials like lithium and proper end-of-life recycling
- **Scalability**: Highly modular and can be deployed incrementally; new technologies like flow batteries and solid-state designs promise improved capacity
- **Cost-Effectiveness**: Costs falling rapidly (80% decrease since 2010); provides multiple value streams including grid stabilization, peak shaving, and emergency backup

## 2. Green Hydrogen Production and Infrastructure
- **Environmental Impact**: Zero-emission fuel when produced with renewable electricity; can decarbonize hard-to-electrify sectors
- **Scalability**: Leverages existing natural gas infrastructure with modifications; production can be distributed or centralized
- **Cost-Effectiveness**: Currently expensive but projected to decrease by 60% by 2030 as electr

In [ ]:
# langchain >= 0.1.17

In [63]:
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# ChatBedrock docs: supports LCEL methods like .assign/.bind.  # :contentReference[oaicite:3]{index=3}

llm = Bedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # inference profile ID/ARN if needed
    region_name="us-east-1",
)

idea_prompt = PromptTemplate.from_template(
"""Step 1:
Challenge: {challenge}
Consider: {considerations}
Brainstorm three innovative sustainable-energy solutions (1–2 lines each). Label 1–3.
A:"""
)
evaluation_prompt = PromptTemplate.from_template(
"""Step 2:
Challenge: {challenge}
Consider: {considerations}
Ideas:
{ideas}

Evaluate each idea for environmental impact, scalability, and cost-effectiveness.
Return a compact table and end with: TOP=<idea #> : <name>."""
)
recommendation_prompt = PromptTemplate.from_template(
"""Step 3:
Based on {ideas} and the evaluation below, produce final recommendations:
- 5 actionable next steps
- 90-day pilot scope
- Rough cost range and risks
Evaluation:
{evaluation}
A:"""
)

# Runnables for each step
ideas_chain         = idea_prompt         | llm | StrOutputParser()
evaluation_chain    = evaluation_prompt   | llm | StrOutputParser()
final_chain         = recommendation_prompt| llm | StrOutputParser()

In [65]:
# Compose sequentially, carrying state forward with `.assign`
sustainable_energy_pipeline = (
    RunnablePassthrough.assign(ideas=ideas_chain)
    .assign(evaluation=evaluation_chain)
    .assign(final_recommendations=final_chain)
)

In [ ]:
# Invoke
out = sustainable_energy_pipeline.invoke({
    "challenge": "sustainable energy solutions",
    "considerations": "environmental impact, scalability, cost-effectiveness",
})
print(out["final_recommendations"])  # get just the final text

ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: messages: Field required